In [ ]:
%%html
<style>
.container {
  width: 100%;
}
</style>

In [ ]:
%load_ext nb_mypy

In [ ]:
import chess
import nbimporter
from Exercise02AI import Exercise02AI

# Aufgabe 03: Minimax (Simplified Evaluation Function)

Dieses Notebook implementiert die Simplified Evaluation Function zur Berechnung des nächsten Zuges im Mittelspiel. Hierbei wird die `minimax`-Funktion der Super-Klasse (`Exercise02AI`) aufgerufen und durch die nun implementierte Simplified Evaluation Function verbessert.

Hierzu werden nachfolgend die in [1] beschriebenen `Piece-Square Tables` definiert. Diese weisen jeder der 64 möglichen Positionen auf dem Schachfeld für alle der Schachfiguren (Bauer, Springer, Läufer, Turm, Dame und König) eine numerische Bewertung in Form einer Ganzzahl zu. Diese Zahl kann sowohl positiv (gute Position für die Figur) oder negativ (schlechte Position für die Figur) sein. Die Tabelle wird als Liste dargestellt. Der König verfügt über zwei verschiedene Listen, eine für das Mittelspiel und eine für das Endspiel.



Die statische Hilfsfunktion `transform_pst` wird im Konstruktor der `Exercise03AI` aufgerufen und dient dazu, eine gegebene PST an der x-Achse zu spiegeln und einen übergebenen Materialwert auf jedes Feld aufzuaddieren.

In [ ]:
class Exercise03AI(Exercise02AI):
    """Chooses middle game moves using minimax algorithm and piece square tables."""

    @staticmethod
    def transform_pst(PST: list[int], material_value: int) -> list[int]:
        """Mirrors a piece square table on the x-axis and adds material values."""
        rows = [PST[i : i + 8] for i in range(0, len(PST), 8)]
        return [field + material_value for row in reversed(rows) for field in row]

Das Dictionary `PIECE_TO_PST` weist jedem Figurentyp die entsprechende Piece-Square Table zu. Hierbei werden die Tabellen für die interne Verarbeitung mithilfe der Funktion `transform_pst` an der x-Achse gespiegelt. Um für die Figur des Königs zwischen Mittel- und Endpiel unterscheiden zu können, existiert zusätzlich das Dictionary `PIECE_TO_PST_KING_ENDGAME`.

Für eine effiziente Evaluierung werden die Materialwerte für die Figuren bereits in die Tabelle mit einberechnet. Hierfür werden die Materialwerte der einzelnen Figuren in *Centipawns* benötigt. Folglich werden die bereits in der Elternklasse gesetzten Werte wie folgt angepasst:

| Figurname  | Materialwert |
|---|---|
| Bauer (pawn) | 100  |
| Springer (knight) | 320  |
| Läufer (bishop) | 330  |
| Turm (rook) | 500  |
| Dame (queen) | 900  |
| König (king) | 0 |

In [ ]:
class Exercise03AI(Exercise03AI):  # type: ignore
    def __init__(self, **kwargs) -> None:
        super().__init__(**kwargs)
        self.is_king_endgame: bool = False
        # Create piece square table mapping (needs to revert tables)
        self.PIECE_TO_PST = {
            chess.PAWN: self.transform_pst(self.PST_PAWN, 100),
            chess.KNIGHT: self.transform_pst(self.PST_KNIGHT, 320),
            chess.BISHOP: self.transform_pst(self.PST_BISHOP, 330),
            chess.ROOK: self.transform_pst(self.PST_ROOK, 500),
            chess.QUEEN: self.transform_pst(self.PST_QUEEN, 900),
            chess.KING: self.transform_pst(self.PST_KING, 0),
        }
        self.PIECE_TO_PST_KING_ENDGAME = {
            **self.PIECE_TO_PST,
            chess.KING: self.transform_pst(self.PST_KING_ENDGAME, 0),
        }
        # Start with normal tables
        self.tables = self.PIECE_TO_PST


Wie auch schon in `Exercise02AI` existiert eine Funktion `reset`, die dazu dient, die Instanz nach einem beendeten Spiel in ihren Ursprungszustand zurückzusetzen.

In [ ]:
class Exercise03AI(Exercise03AI):  # type: ignore
    def reset(self) -> None:
        """Resets all internal variables."""
        super().reset()
        self.is_king_endgame = False
        self.tables = self.PIECE_TO_PST

Nachgehend werden für alle Figuren die benötigten Piece-Square Tables definiert sowie kurz beschrieben.
Anhand der Piece-Square Table der Bauern `PST_PAWN`, die im Folgenden definiert ist, wird ersichtlich, dass Bauern auf den Feldern `f2`, `g2` und `h2` (in symmetrischer Weise auch auf `a2`, `b2` und `c2`) Boni (positive Bewertung) erhalten, während sie auf den Felder `f3` und `g3` Abzüge (negative Bewertungen) erhalten. Dies führt dazu, dass die KI zentrale Bauern nicht stillstehen lässt und diese bereits zu Beginn des Spiels vorwärts bewegt.

In [ ]:
class Exercise03AI(Exercise03AI):  # type: ignore
    PST_PAWN = [
        0,  0,  0,  0,  0,  0,  0,  0,
       50, 50, 50, 50, 50, 50, 50, 50,
       10, 10, 20, 30, 30, 20, 10, 10,
        5,  5, 10, 25, 25, 10,  5,  5,
        0,  0,  0, 20, 20,  0,  0,  0,
        5, -5,-10,  0,  0,-10, -5,  5,
        5, 10, 10,-20,-20, 10, 10,  5,
        0,  0,  0,  0,  0,  0,  0,  0
    ]

Springer sollen sich vor allem im Mittelfeld aufhalten, da nur so alle acht Zug-Felder genutzt werden können. Auf den Feldern `d4` und `d5` sowie `e4` und `e5` erhalten diese hohe Werte. Am Rand und vor allem in den vier Ecken erhalten Springer hohe Abzüge, da hier die Möglichkeiten der Bewegung sehr stark eingeschränkt sind.

In [ ]:
class Exercise03AI(Exercise03AI):  # type: ignore
    PST_KNIGHT = [
        -50,-40,-30,-30,-30,-30,-40,-50,
        -40,-20,  0,  0,  0,  0,-20,-40,
        -30,  0, 10, 15, 15, 10,  0,-30,
        -30,  5, 15, 20, 20, 15,  5,-30,
        -30,  0, 15, 20, 20, 15,  0,-30,
        -30,  5, 10, 15, 15, 10,  5,-30,
        -40,-20,  0,  5,  5,  0,-20,-40,
        -50,-40,-30,-30,-30,-30,-40,-50
    ]

Für Läufer gelten ähnliche Regelungen wie für Springer. Auch diese sollten möglichst in der Mitte des Spielfelds stehen und weder am Rand noch in den Ecken plaziert werden.

In [ ]:
class Exercise03AI(Exercise03AI):  # type: ignore
    PST_BISHOP = [
        -20,-10,-10,-10,-10,-10,-10,-20,
        -10,  0,  0,  0,  0,  0,  0,-10,
        -10,  0,  5, 10, 10,  5,  0,-10,
        -10,  5,  5, 10, 10,  5,  5,-10,
        -10,  0, 10, 10, 10, 10,  0,-10,
        -10, 10, 10, 10, 10, 10, 10,-10,
        -10,  5,  0,  0,  0,  0,  5,-10,
        -20,-10,-10,-10,-10,-10,-10,-20
    ]

Türme haben nur kleine Differenzen in der Bewertung. So stehen diese auf den Feldern `d1` und `e1` sowie auf den 7. Linie (`a7` bis `h7`) besonders gut. An den Vertikalen am Rand (Linie `a` und `h`) stehen diese schlechter.

In [ ]:
class Exercise03AI(Exercise03AI):  # type: ignore
    PST_ROOK = [
        0,  0,  0,  0,  0,  0,  0,  0,
        5, 10, 10, 10, 10, 10, 10,  5,
       -5,  0,  0,  0,  0,  0,  0, -5,
       -5,  0,  0,  0,  0,  0,  0, -5,
       -5,  0,  0,  0,  0,  0,  0, -5,
       -5,  0,  0,  0,  0,  0,  0, -5,
       -5,  0,  0,  0,  0,  0,  0, -5,
        0,  0,  0,  5,  5,  0,  0,  0
    ]

Die Dame kann durch ihre hohe Anzahl an Zug- und Schlag-Richtungen in der Mitte des Spielfelds sehr viel abdecken, sollte also dort ausgerichtet sein. Auch sie sollte weder am Rand noch in den Ecken platziert werden.

In [ ]:
class Exercise03AI(Exercise03AI):  # type: ignore
    PST_QUEEN = [
        -20,-10,-10, -5, -5,-10,-10,-20,
        -10,  0,  0,  0,  0,  0,  0,-10,
        -10,  0,  5,  5,  5,  5,  0,-10,
         -5,  0,  5,  5,  5,  5,  0, -5,
          0,  0,  5,  5,  5,  5,  0, -5,
        -10,  5,  5,  5,  5,  5,  0,-10,
        -10,  0,  5,  0,  0,  0,  0,-10,
        -20,-10,-10, -5, -5,-10,-10,-20
    ]

Einzig für den König gibt es zwei Piece-Square Tables. Die Folgende ist für das Mittelspiel, die Nächste für das Endspiel. Im Mittelspiel soll durch die Wertung verhindert werden, dass der König zu früh in die Mitte des Spielfelds zieht und von anderen Figuren angegriffen werden kann. Auf der Grundlinie ist er von seinen anderen Figuren geschützt.

In [ ]:
class Exercise03AI(Exercise03AI):  # type: ignore
    PST_KING = [
        -30,-40,-40,-50,-50,-40,-40,-30,
        -30,-40,-40,-50,-50,-40,-40,-30,
        -30,-40,-40,-50,-50,-40,-40,-30,
        -30,-40,-40,-50,-50,-40,-40,-30,
        -20,-30,-30,-40,-40,-30,-30,-20,
        -10,-20,-20,-20,-20,-20,-20,-10,
         20, 20,  0,  0,  0,  0, 20, 20,
         20, 30, 10,  0,  0, 10, 30, 20
    ]

Im Endspiel, das im Folgenden genauer definiert ist, sollte der König die Grundlinie verlassen und ins Zentrum des Spielfelds ziehen. Dies ist aufgrund seiner geringen Beweglichkeit wichtig, da er dort mehr Möglichkeiten der Offensive hat.

In [ ]:
class Exercise03AI(Exercise03AI):  # type: ignore
    PST_KING_ENDGAME = [
        -50,-40,-30,-20,-20,-30,-40,-50,
        -30,-20,-10,  0,  0,-10,-20,-30,
        -30,-10, 20, 30, 30, 20,-10,-30,
        -30,-10, 30, 40, 40, 30,-10,-30,
        -30,-10, 30, 40, 40, 30,-10,-30,
        -30,-10, 20, 30, 30, 20,-10,-30,
        -30,-30,  0,  0,  0,  0,-30,-30,
        -50,-30,-30,-30,-30,-30,-30,-50
    ]

## Prüfung der Endspielkonditionen

Um bei der Simplified Evaluation Function die richtige Königstabelle auszuwählen ist es notwendig zu prüfen, ob sich das aktuelle Spiel im Mittel- oder Endspiel befindet. Das Endspiel wird dabei definiert als ein Spiel bei dem entweder:

1. beide Seiten keine Dame mehr besitzen oder
2. jede Seite die eine Dame hat, keinen Turm und maximal einen Läufer oder Springer besitzt.

**Hinweis**: Diese Definition der Endspielkonditionen bezieht sich nur auf die Simplified Evaluation Function. Allgemein ist das Endspiel (implementiert durch die Syzygy Endgame Tablebases) erreicht, sobald 5 oder weniger Figuren auf dem Spielbrett vorhanden sind.

Die Funktion `check_king_endgame` implementiert die oben genannten Bedingungen und prüft, ob sie für ein gegebenes Board erfüllt sind.

In [ ]:
class Exercise03AI(Exercise03AI):  # type: ignore
    def check_king_endgame(self, board: chess.Board) -> bool:
        """Checks if the king endgame tables should be used."""
        return all(
            (
                (
                    not board.pieces(chess.QUEEN, color)
                    or (
                        len(board.pieces(chess.BISHOP, color))
                        + len(board.pieces(chess.KNIGHT, color))
                        < 2
                    )
                    and not board.pieces(chess.ROOK, color)
                )
                for color in chess.COLORS
            )
        )

## Evaluierungsfunktion

Die Funktion `calculate_pst_value` nimmt als Argument einen Figurentyp (`piece`), eine Farbe (`color`) und eine Menge von Positionen (`positions`). Anschließend wird für jede Position der Wert in der entsprechenden Piece-Square Table ermittelt und die Summe dieser Werte zurückgegeben. Für schwarze Figuren wird das Board dabei gespiegelt und das Resultat negiert.

In [ ]:
class Exercise03AI(Exercise03AI):  # type: ignore
    def calculate_pst_value(
        self,
        piece: chess.PieceType,
        color: chess.Color,
        positions: chess.SquareSet
    ) -> int:
        """Calculates the piece-square table values for a given piece, color and and optional position."""
        if color == chess.WHITE:
            return sum([self.tables[piece][i] for i in positions])
        else:
            return -sum(
                [(self.tables[piece][chess.square_mirror(i)]) for i in positions]
            )

Die folgenden Funktionen der Elternklasse (`Exercise02AI`) wurden überschrieben:

- `full_evaluate`
- `incremental_evaluate`

Die Funktion `full_evaluate` berechnet den Wert eines Boards von Grund auf neu und geht dabei wie folgt vor:

1. Es wird für jede Figur und für jede Farbe mithilfe der Funktion `calculate_pst_value` die Bewertung der Figur in Centipawns ermittelt.
2. Es wird die Summe über alle Bewertungen gebildet. Je größer die Summe, desto besser ist Weiß auf dem Feld positioniert, umgekehrt ist ein negatives Ergebnis ein Indikator dafür, dass Schwarz besser auf dem Board positioniert ist.
3. Die Summe aller Bewertungen wird zurückgegeben.

Der Rückgabewert der Funktion beschreibt also die Kombination des reinen Materialwertes der Figuren beider Spieler und der Positionsbewertung durch die `Piece-Square Tables`.

In [ ]:
class Exercise03AI(Exercise03AI):  # type: ignore
    def full_evaluate(self, board: chess.Board) -> int:
        """Evaluates a given board."""
        # Get if pieces standing well or pieces standing badly by calculating with the Piece-Square Tables
        return sum(
            self.calculate_pst_value(piece_type, color, board.pieces(piece_type, color))
            for piece_type in chess.PIECE_TYPES
            for color in chess.COLORS
        )

Die Funktion `incremental_evaluate` berechnet die Änderungen der Bewertung, wenn der gegebene Zug (`next_move`) auf das übergebene Board (`board`) angewendet wird. Diese Änderung wird zurückgegeben. Hierbei werden die folgenden Fälle beachtet:

- Umwandlung eines Bauern in eine höherwertige Figur
- Veränderung der Position der gezogenen Figur
- Schlagen einer Figur (auch durch einen En Passant Zug)
- Lange (große) und kurze (kleine) Rochade

Wird ein Bauer in eine höherwertige Figur umgewandelt, so wird die alte Positionsbewertung des gezogenen Bauern von der Bewertung der umgewandelten Figur subtrahiert. Das Ergebnis wird in der Variable `change` gespeichert.
War der letzte Zug keine Umwandlung, so wird die alte Bewertung der gezogenen Figur von ihrer neuen Bewertung abgezogen. Das Ergebnis wird ebenfalls in `change` gespeichert.

Handelt es sich bei dem letzten Zug um eine lange Rochade, so ändert sich die Positionsbewertung des Turms. Hierzu wird der Materialwert eines Turms auf `change` addiert, falls Weiß am Zug ist, oder von `change` subtrahiert, falls Schwarz am Zug ist. Bei einer kurzen Rochade entsteht keine Veränderung der Positionsbewertung durch den bewegten Turm.

Falls während des letzten Zuges eine Figur geschlagen wurde, wird die alte Bewertung der geschlagenen Figur errechnet und von `change` subtrahiert. Hierbei wird eine separate Abfrage verwendet, um zu erkennen, ob es sich bei dem letzten Zug um einen En Passant Zug handelt.

Zurückgegeben wird schlussendlich die Summe von `change`.

In [ ]:
class Exercise03AI(Exercise03AI):  # type: ignore
    def incremental_evaluate(
        self,
        board: chess.Board,
        last_evaluation: int,
        next_move: chess.Move
    ) -> int:
        """Returns an incrementally calculated evaluation of the given board."""
        if promotion_piece_type := next_move.promotion:
            # Calculate change caused by piece promotion
            old_piece_pst = self.calculate_pst_value(
                chess.PAWN, board.turn, [next_move.from_square]
            )
            new_piece_pst = self.calculate_pst_value(
                promotion_piece_type, board.turn, [next_move.to_square]
            )
            change = new_piece_pst - old_piece_pst
        else:
            # Calculate change caused by piece move
            piece_type = board.piece_type_at(next_move.from_square)
            old_pst_value = self.calculate_pst_value(
                piece_type, board.turn, [next_move.from_square]
            )
            new_pst_value = self.calculate_pst_value(
                piece_type, board.turn, [next_move.to_square]
            )
            change = new_pst_value - old_pst_value

        if captured_piece_type := board.piece_type_at(next_move.to_square):
            # Calculate additional change caused by piece capture
            captured_piece_pst = self.calculate_pst_value(
                captured_piece_type, not board.turn, [next_move.to_square]
            )
            change -= captured_piece_pst
        elif board.is_en_passant(next_move):
            # Calculate additional change caused by en passant pawn capture
            pawn_double_move = board.peek()
            captured_pawn_pst = self.calculate_pst_value(
                chess.PAWN, not board.turn, [pawn_double_move.to_square]
            )
            change -= captured_pawn_pst
        elif board.is_queenside_castling(next_move):
            # Add changed rook value caused by long castling (short castling does not change rook value and can be ignored)
            change += 5 * (1 if board.turn else -1)

        return last_evaluation + change

## Berechnung des besten Zuges

Die Funktion `get_endgame_evaluation_change` berechnet die Differenz der Bewertung die beim Übergang in das Königs-Endspiel oder umgekehrt beim Übergang zurück in das Königs-Mittelspiel stattfindet.

In [ ]:
class Exercise03AI(Exercise03AI):  # type: ignore
    def get_endgame_evaluation_change(self, board: chess.Board) -> int:
        """Calculates the change of the evaluation when switching endgame status."""
        self.is_king_endgame: bool  # mypy type declaration
        old_rating_king_white = self.calculate_pst_value(
            chess.KING, chess.WHITE, board.pieces(chess.KING, chess.WHITE)
        )
        old_rating_king_black = self.calculate_pst_value(
            chess.KING, chess.BLACK, board.pieces(chess.KING, chess.BLACK)
        )
        self.is_king_endgame = not self.is_king_endgame
        self.tables = (
            self.PIECE_TO_PST_KING_ENDGAME
            if self.is_king_endgame
            else self.PIECE_TO_PST
        )
        new_rating_king_white = self.calculate_pst_value(
            chess.KING, chess.WHITE, board.pieces(chess.KING, chess.WHITE)
        )
        new_rating_king_black = self.calculate_pst_value(
            chess.KING, chess.BLACK, board.pieces(chess.KING, chess.BLACK)
        )
        return (
            new_rating_king_white
            - old_rating_king_white
            + new_rating_king_black
            - old_rating_king_black
        )

Die Funktion `get_next_middle_game_move` berechnet auf einem gegebenen Board den nächsten besten Zug. Zusätzlich zur bereits vorhandenen Funktionalität der Elternklasse wird hier überprüft, ob sich der Endspielstatus geändert hat. Falls zutreffend, wird die letzte gespeicherte Beurteilung mithilfe der Funktion `get_endgame_evaluation_change` entsprechend angepasst, da der König über eine separate `Piece-Square Table` für das Endspiel verfügt. Aus Performance-Gründen wird der Endspielstatus innerhalb der Minimax Funktion nicht jedes Mal erneut überprüft.

**Hinweis**: `self.last_evaluation` kann in der Theorie den Wert `None` annehmen und hier einen Fehler auslösen. Dies ist aber nur zu Beginn des Mittelspiels der Fall. Der Status des Spiels zum Endspiel ändert sich aber erst im späteren Spielverlauf, daher tritt der Fall in der Praxis nicht ein und wird folglich nicht geprüft.

In [ ]:
class Exercise03AI(Exercise03AI):  # type: ignore
    def get_next_middle_game_move(self, board: chess.Board) -> chess.Move:
        """Gets the best next move."""
        self.last_evaluation: int  # type annotation for mypy
        if self.is_king_endgame != self.check_king_endgame(board):
            self.last_evaluation += self.get_endgame_evaluation_change(board)
        return super().get_next_middle_game_move(board)

## Debugging Bereich

Die folgenden Zellen enthalten Unit-Tests der oben implementierten Funktionen.

In [ ]:
from AIBaseClass import ChessAI
import Exercise02AI as Exercise02AI_

In [ ]:
# Create player and board
unit_test_player = Exercise03AI(player_name="Ex03AI", search_depth=3)
board = chess.Board("5rk1/1b3p2/8/3p4/3p2P1/2Q4B/5P1K/R3R3 b - - 0 36")
board

Um festzustellen, ob sich das Spiel im Endspiel (Definition s. o.) befindet, wurde die Funktion `check_king_endgame` entwickelt. Um deren Funktionsweise zu validieren, wird bei dem gegebenen Board überprüft, ob richtig erkannt wird, dass sich das Spiel erst nach dem Schlagen der Dame im Endspiel befindet.

In [ ]:
# Test endgame check
def test_endgame_check(unit_test_player: ChessAI, board: chess.Board):
    is_endgame_1 = unit_test_player.check_king_endgame(board)
    board.push(chess.Move.from_uci("d4c3"))  # white queen capture
    is_endgame_2 = unit_test_player.check_king_endgame(board)
    board.pop()
    print(f"Endgame 1: {is_endgame_1} - Endgame 2: {is_endgame_2}")
    assert is_endgame_1 is False, "Endgame status should be False!"
    assert is_endgame_2 is True, "Endgame status should be True!"

In [ ]:
test_endgame_check(unit_test_player, board)

Die Funktion `calculate_pst_value` berechnet für eine gegebene Spielfigur und eine Liste von Spielfeldern die Evaluierung. Um diese Funktion zu testen, wird für Weiß der Wert des Bauers an der Position F2 überprüft, welcher laut Piece-Square Table $10$ sein muss. Für den schwarzen Spieler wird der Läufer auf B7 überprüft, wobei dessen Position über `chess.square_mirror` gespiegelt wird. Der erwartete Wert beträgt hierbei $5$, wird allerdings negiert, da es sich um die Bewertung für den schwarzen Spieler handelt.

In [ ]:
# Test pst calculation
def test_pst_calculation(unit_test_player: ChessAI, board: chess.Board):
    pst_value_white = unit_test_player.calculate_pst_value(
        piece=chess.PAWN, color=chess.WHITE, positions=[chess.F2]
    )
    pst_value_black = unit_test_player.calculate_pst_value(
        piece=chess.BISHOP, color=chess.BLACK, positions=[chess.B7]
    )
    print(f"PST Values: White {pst_value_white} - Black {pst_value_black}")
    assert pst_value_white == 110, "PST calculation for white does not match expected value!"
    assert pst_value_black == -335, "PST calculation for black does not match expected value!"

In [ ]:
test_pst_calculation(unit_test_player, board)

Um die Berechnung der PSTs zu validieren, wird für die insgesamt 13 Figuren auf dem aktuellen Board die Bewertung überprüft. Da die Materialwerte nun in Centipawns berechnet werden, lautet der neue Wert der `full_evaluate` aus `Exercise02AI` nicht mehr $13$, sondern $1300$. Ausgehend von dieser Basisbewertung werden nun die folgenden Positionsbewertungen mit eingerechnet:

|         | Bauer | Läufer | Turm | Dame | König |
|---------|-------|--------|------|------|-------|
| Weiß    | 10    | -10    | 5    | 5    | 20    |
| Schwarz | 55    | 5      | -    | -    | 30    |

Die Stellung des schwarzen Spielers wird also um 60 Punkte besser bewertet, welche von der Basisbewertung subtrahiert werden. Folglich lautet der erwartete Wert $1240$.

In [ ]:
# Test full evaluation
def test_full_evaluation(unit_test_player: ChessAI, board: chess.Board):
    full_eval = unit_test_player.full_evaluate(board)
    print(f"Full Evaluation: {full_eval}")
    assert full_eval == 1240, "Full evaluation does not match expected value!"

In [ ]:
test_full_evaluation(unit_test_player, board)

Im Folgenden wird die Funktion `incremental_evaluate` getestet, wobei der schwarze Spieler mit seinem Bauer die Dame des weißen Spielers schlägt. Der verlorene Materialwert der Dame beträgt 900 Centipawns, ihr Stellungswert von $5$ geht ebenfalls verloren. Da der Positionswert des schwarzen Bauern sich ebenfalls um fünf Punkte reduziert, entspricht der erwartete Rückgabewert der Funktion dem Materialwert der Dame.

In [ ]:
# Test incremental evaluation
def test_incremental_evaluation(unit_test_player: ChessAI, board: chess.Board):
    next_move = chess.Move.from_uci("d4c3")  # white queen capture
    inc_eval = unit_test_player.incremental_evaluate(board, 0, next_move)
    print(f"Incremental Evaluation: {inc_eval}")
    assert inc_eval == -900, "Incremental evaluation does not match expected value!"

In [ ]:
test_incremental_evaluation(unit_test_player, board)

Nachdem die vollständige und die inkrementelle Evaluierungsfunktion getestet wurden, wird nun das Zusammenspiel beider Funktionen getestet. Hierbei wird überprüft, ob das Ergebnis der `incremental_evaluate` Funktion mit einem gebenenen `next_move` dem der Funktion `full_evaluate` entspricht, die ein Board erhält, auf welches eben dieser Zug angewendet wurde.

In [ ]:
# Test evaluation
def test_evaluation(unit_test_player: ChessAI, board: chess.Board):
    next_move = chess.Move.from_uci("d4c3")  # white queen capture
    full_eval, inc_eval = 1240, -900
    evaluation = unit_test_player.incremental_evaluate(board, full_eval, next_move)
    board.push(next_move)
    new_full_eval = unit_test_player.full_evaluate(board)
    board.pop()
    print(f"Full Evaluation: {full_eval}")
    print(f"Incremental Evaluation: {inc_eval}")
    print(f"Evaluation: {evaluation}")
    print(f"New Full Evaluation: {new_full_eval}")
    assert evaluation == full_eval + inc_eval, "Evaluation does not match expected sum!"
    assert new_full_eval == evaluation, "Incremental and full evaluation are different!"

In [ ]:
test_evaluation(unit_test_player, board)

Der König verfügt über separate Tabellen für das Königs-Mittel- und Königs-Endspiel. Um den Übergang zu testen, wird die Funktion `get_endgame_evaluation_change` überprüft. Für das aktuelle Board beträgt die Differenz beider Bewertungen der Könige $-10$ im Mittelspiel und $0$ im Endspiel. Die zu testende Funktion wird zweimal aufgerufen, wobei zunächst der Übergang vom Königs-Mittel- in das Königs-Endspiel simuliert wird. Die erwartete Differenz beträgt hierbei $0 - (-10) = 10$. Falls ein Übergang zurück in das Königs-Mittelspiel stattfindet, wird der negierte Differenzwert erwartet, was mit dem zweiten Funktionsaufruf getestet wird.

In [ ]:
# Test endgame change calculation
def test_endgame_change_calc(unit_test_player: ChessAI, board: chess.Board):
    change_to_endgame = unit_test_player.get_endgame_evaluation_change(board)
    change_from_endgame = unit_test_player.get_endgame_evaluation_change(board)
    print(f"Change to endgame: {change_to_endgame}")
    print(f"Change from endgame: {change_from_endgame}")
    assert change_to_endgame == 10, "Endgame change calculation does not match expected value!"
    assert change_from_endgame == -10, "Endgame change calculation does not match expected value!"

In [ ]:
test_endgame_change_calc(unit_test_player, board)

Um die Minimax-Funktion zu testen, werden sowohl die Evaluierung als auch der zurückgegebene Zug und die Anzahl der berechneten Nodes überprüft. Auf das gegebene Board wird nun der Zug angewendet, welcher die weiße Dame schlägt. Dadurch beträgt die erwartete Evaluierung $325$. Die Anzahl der Nodes, die untersucht werden sollen, beträgt $14377$ und dient dazu, Änderungen in der Berechnung aufzuzeigen.

In [ ]:
Exercise02AI_.test_minimax(
    unit_test_player,
    board,
    current_evaluation=1240,
    expected_evaluation=325,
    expected_move="d4c3",
    expected_nodes=14377,
)

Um die Funktion `get_next_middle_game_move` zu testen, wird überprüft, ob diese korrekterweise den Zug zurückgibt, in welchem die Dame durch den schwarzen Bauern geschlagen wird und die Anzahl der berechneten Knoten analog zum Minimax-Unittest $14377$ entspricht.

In [ ]:
Exercise02AI_.test_next_move(
    unit_test_player,
    board,
    expected_move="d4c3",
    expected_nodes=14377,
)

Da die `reset`-Funktion der `Exercise03AI` erweitert wurde, wird im Folgenden auch der Unittest der Funktion angepasst. Überprüft wird hierbei, ob sowohl die letzte Evaluierung, als auch der Status des König-Endspiels und die Piece-Square Tables zurückgesetzt wurden.

In [ ]:
# Test reset function
def test_reset(unit_test_player: ChessAI, _: chess.Board):
    unit_test_player.reset()
    assert unit_test_player.last_evaluation is None, "Reset was not successful! (last_evaluation)"
    assert unit_test_player.is_king_endgame is False, "Reset was not successful! (is_king_endgame)"
    assert unit_test_player.tables == unit_test_player.PIECE_TO_PST, "Reset was not successful! (tables)"

In [ ]:
test_reset(unit_test_player, board)

In den folgenden Unittests wird weiterhin die Funktionsweise der Funktion `calculate_pst_value` getestet, wobei jedes Mal leicht unterschiedliche Boards eingesetzt werden, um einen definierten Wert zu berechnen.

In [ ]:
def test_pst(unit_test_player: ChessAI, board: chess.Board, expected_value: int):
    rating = sum(
        unit_test_player.calculate_pst_value(
            piece_type, color, board.pieces(piece_type, color)
        )
        for piece_type in chess.PIECE_TYPES
        for color in chess.COLORS
    )
    print(f"PST evaluation: {rating}")
    assert rating == expected_value, f"PST evaluation does not match expected value of {expected_value}!"

Im ersten Testdurchlauf stehen sich beide Könige gegenüber, wobei Weiß über eine zusätzliche Dame verfügt. Alle Figuren auf dem Spielfeld haben eine Positionsbewertung von $0$, wodurch der erwartete Rückgabewert der Funktion mit $900$ dem Materialwert der Dame entspricht.

In [ ]:
board_pst = chess.Board("4k3/8/8/8/Q7/8/8/4K3 w - - 0 1")
# display(board_pst)
test_pst(unit_test_player, board_pst, expected_value=900)

Im Vergleich zum vorherigen Board ist die Dame von A4 auf H4 umgezogen, wodurch ihre Positionsbewertung um $5$ verschlechtert wurde.

In [ ]:
board_pst = chess.Board("4k3/8/8/8/7Q/8/8/4K3 w - - 0 1")
# display(board_pst)
test_pst(unit_test_player, board_pst, expected_value=895)

Der weiße Spieler zieht die Dame von H4 auf H5, wobei der schwarze König Schach gesetzt wird. Die Positionsbewertung der Figuren ändert sich hierdurch nicht.

In [ ]:
board_pst = chess.Board("4k3/8/8/7Q/8/8/8/4K3 b - - 0 1")
# display(board_pst)
test_pst(unit_test_player, board_pst, expected_value=895)

Im Folgenden werden die Rollen der Spieler getauscht. Es ist also nun Schwarz mit einer Dame im Vorteil, deren Positionsbewertung auf A5 $0$ beträgt, wodurch die erwartete Bewertung dem negativen Materialwert der Dame entspricht.

In [ ]:
board_pst = chess.Board("4k3/8/8/q7/8/8/8/4K3 w - - 0 1")
# display(board_pst)
test_pst(unit_test_player, board_pst, expected_value=-900)

Die schwarze Dame ändert nun ihre Position von A5 auf H5, wodurch ihre Positionsbewertung um $5$ Centipawns reduziert wird.

In [ ]:
board_pst = chess.Board("4k3/8/8/7q/8/8/8/4K3 w - - 0 1")
# display(board_pst)
test_pst(unit_test_player, board_pst, expected_value=-895)

Mit einer weiteren Positionsänderung von H5 auf H6 wird die Positionsbewertung der schwarzen Dame erneut um $5$ Centipawns reduziert.

In [ ]:
board_pst = chess.Board("4k3/8/7q/8/8/8/8/4K3 w - - 0 1")
# display(board_pst)
test_pst(unit_test_player, board_pst, expected_value=-890)